<a href="https://airlineflightpriceprediction.streamlit.app/">Flight_Price_prediction_app </a>

In [1]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder,HashingEncoder
from sklearn.preprocessing import OneHotEncoder

import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector,RFE

from sklearn.model_selection import cross_validate

from sklearn.model_selection import GridSearchCV
import joblib as jb

In [2]:
ML = pd.read_csv('cleaned_data.csv')

In [3]:
ML

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Price,day_of_journey,month_of_journey,count_journey_days,dep_day_part,arrival_day_part,check-in baggage included,In-flight meal,Change airports,Business class,layover,stop_points
0,Air India,2019-05-01,Kolkata,Banglore,05:50,13:15,7.42,2,7662,1,5,0.0,early morning,afternoon,1,1,0,0,0,"[' IXR ', ' BBI ']"
1,Jet Airways,2019-06-09,Delhi,Cochin,09:25,04:25,19.00,2,13882,9,6,1.0,morning,mid night,1,1,0,0,0,"[' LKO ', ' BOM ']"
2,IndiGo,2019-05-12,Kolkata,Banglore,18:05,23:30,5.42,1,6218,12,5,0.0,evening,night,1,1,0,0,0,[' NAG ']
3,IndiGo,2019-03-01,Banglore,New Delhi,16:50,21:35,4.75,1,13302,1,3,0.0,afternoon,evening,1,1,0,0,0,[' NAG ']
4,SpiceJet,2019-06-24,Kolkata,Banglore,09:00,11:25,2.42,0,3873,24,6,0.0,morning,morning,1,1,0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,Air Asia,2019-04-09,Kolkata,Banglore,19:55,22:25,2.50,0,4107,9,4,0.0,evening,evening,1,1,0,0,0,[]
10316,Air India,2019-04-27,Kolkata,Banglore,20:45,23:20,2.58,0,4145,27,4,0.0,evening,night,1,1,0,0,0,[]
10317,Jet Airways,2019-04-27,Banglore,Delhi,08:20,11:20,3.00,0,7229,27,4,0.0,morning,morning,1,1,0,0,0,[]
10318,Vistara,2019-03-01,Banglore,New Delhi,11:30,14:10,2.67,0,12648,1,3,0.0,morning,afternoon,1,1,0,0,0,[]


# preparing to ML

In [4]:
ML.drop(['Date_of_Journey','Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [5]:
ML.duplicated().sum()

109

In [6]:
ML.drop_duplicates(inplace=True)

In [7]:
ML.reset_index(drop=True,inplace=True)

In [8]:
ML.shape

(10211, 17)

In [9]:
for col in ML.columns:
   
        print(f'{col} {ML[col].nunique()}')
    

Airline 12
Source 5
Destination 6
Duration 364
Total_Stops 5
Price 1840
day_of_journey 10
month_of_journey 4
count_journey_days 3
dep_day_part 6
arrival_day_part 6
check-in baggage included 2
In-flight meal 2
Change airports 2
Business class 2
layover 4
stop_points 98


In [10]:
for col in ML.columns:
    print(ML[col].value_counts(),'\n','_'*50)

Airline
Jet Airways                          3609
IndiGo                               1951
Air India                            1653
Multiple carriers                    1196
SpiceJet                              814
Vistara                               456
Air Asia                              315
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64 
 __________________________________________________
Source
Delhi       4345
Kolkata     2814
Banglore    1995
Mumbai       689
Chennai      368
Name: count, dtype: int64 
 __________________________________________________
Destination
Cochin       4345
Banglore     2814
Delhi        1232
New Delhi     763
Hyderabad     689
Kolkata       368
Name: count, dtype: int64 
 __________________________________________________
Duration
2.83     517
1.50     380
2.75

In [11]:
ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10211 entries, 0 to 10210
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Airline                    10211 non-null  object 
 1   Source                     10211 non-null  object 
 2   Destination                10211 non-null  object 
 3   Duration                   10211 non-null  float64
 4   Total_Stops                10211 non-null  int64  
 5   Price                      10211 non-null  int64  
 6   day_of_journey             10211 non-null  int64  
 7   month_of_journey           10211 non-null  int64  
 8   count_journey_days         10211 non-null  float64
 9   dep_day_part               10211 non-null  object 
 10  arrival_day_part           10211 non-null  object 
 11  check-in baggage included  10211 non-null  int64  
 12  In-flight meal             10211 non-null  int64  
 13  Change airports            10211 non-null  int

In [12]:
onh = ['Source','Destination' , 'dep_day_part' , 'arrival_day_part']

In [13]:
binary = ['Airline','stop_points']

In [14]:
X = ML.drop('Price',axis=1)

In [15]:
X

,Airline,Source,Destination,Duration,Total_Stops,day_of_journey,month_of_journey,count_journey_days,dep_day_part,arrival_day_part,check-in baggage included,In-flight meal,Change airports,Business class,layover,stop_points
0,Air India,Kolkata,Banglore,7.42,2,1,5,0.0,early morning,afternoon,1,1,0,0,0,"[' IXR ', ' BBI ']"
1,Jet Airways,Delhi,Cochin,19.00,2,9,6,1.0,morning,mid night,1,1,0,0,0,"[' LKO ', ' BOM ']"
2,IndiGo,Kolkata,Banglore,5.42,1,12,5,0.0,evening,night,1,1,0,0,0,[' NAG ']
3,IndiGo,Banglore,New Delhi,4.75,1,1,3,0.0,afternoon,evening,1,1,0,0,0,[' NAG ']
4,SpiceJet,Kolkata,Banglore,2.42,0,24,6,0.0,morning,morning,1,1,0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10206,Air Asia,Kolkata,Banglore,2.50,0,9,4,0.0,evening,evening,1,1,0,0,0,[]
10207,Air India,Kolkata,Banglore,2.58,0,27,4,0.0,evening,night,1,1,0,0,0,[]
10208,Jet Airways,Banglore,Delhi,3.00,0,27,4,0.0,morning,morning,1,1,0,0,0,[]
10209,Vistara,Banglore,New Delhi,2.67,0,1,3,0.0,morning,afternoon,1,1,0,0,0,[]


In [16]:
y= ML['Price']

In [17]:
y

0         7662
1        13882
2         6218
3        13302
4         3873
         ...  
10206     4107
10207     4145
10208     7229
10209    12648
10210    11753
Name: Price, Length: 10211, dtype: int64

In [18]:
X.columns

Index(['Airline', 'Source', 'Destination', 'Duration', 'Total_Stops',
       'day_of_journey', 'month_of_journey', 'count_journey_days',
       'dep_day_part', 'arrival_day_part', 'check-in baggage included',
       'In-flight meal', 'Change airports', 'Business class', 'layover',
       'stop_points'],
      dtype='object')

In [19]:
X.describe()

,Duration,Total_Stops,day_of_journey,month_of_journey,count_journey_days,check-in baggage included,In-flight meal,Change airports,Business class,layover
count,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000
mean,10.483595,0.806777,13.443835,4.725198,0.394477,0.968857,0.817158,0.000686,0.000392,0.004113
std,8.303614,0.660022,8.485469,1.154598,0.495528,0.173712,0.386556,0.026175,0.019789,0.091685
min,1.250000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.830000,0.000000,6.000000,4.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,8.420000,1.000000,12.000000,5.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,15.170000,1.000000,21.000000,6.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,47.670000,4.000000,27.000000,6.000000,2.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [20]:
Airlines = ML['Airline'].unique()
Airlines

array(['Air India', 'Jet Airways', 'IndiGo', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [21]:
Sources = ML['Source'].unique()
Sources

array(['Kolkata', 'Delhi', 'Banglore', 'Chennai', 'Mumbai'], dtype=object)

In [22]:
ML[['Source','Destination']].value_counts()

Source    Destination
Delhi     Cochin         4345
Kolkata   Banglore       2814
Banglore  Delhi          1232
          New Delhi       763
Mumbai    Hyderabad       689
Chennai   Kolkata         368
Name: count, dtype: int64

In [23]:
ML[['Airline','Source','Destination']].value_counts()

Airline                            Source    Destination
Jet Airways                        Delhi     Cochin         1438
                                   Kolkata   Banglore       1250
Multiple carriers                  Delhi     Cochin         1196
IndiGo                             Delhi     Cochin          705
Air India                          Delhi     Cochin          703
                                   Kolkata   Banglore        501
IndiGo                             Kolkata   Banglore        405
Jet Airways                        Banglore  Delhi           370
IndiGo                             Banglore  Delhi           348
Jet Airways                        Banglore  New Delhi       337
SpiceJet                           Kolkata   Banglore        300
Jet Airways                        Mumbai    Hyderabad       214
IndiGo                             Mumbai    Hyderabad       195
Vistara                            Kolkata   Banglore        183
IndiGo                           

In [24]:
source_to_dest = list(ML[['Source','Destination']].value_counts().index)
source_to_dest

[('Delhi', 'Cochin'),
 ('Kolkata', 'Banglore'),
 ('Banglore', 'Delhi'),
 ('Banglore', 'New Delhi'),
 ('Mumbai', 'Hyderabad'),
 ('Chennai', 'Kolkata')]

In [25]:
Source_dest = dict()
for source in Sources:
    dest = []
    for s , d in source_to_dest:
        if source == s:
            dest.append(d)
    Source_dest[source] =dest

In [26]:
Source_dest

{'Kolkata': ['Banglore'],
 'Delhi': ['Cochin'],
 'Banglore': ['Delhi', 'New Delhi'],
 'Chennai': ['Kolkata'],
 'Mumbai': ['Hyderabad']}

In [27]:
Destinations = ML['Destination'].unique()
Destinations

array(['Banglore', 'Cochin', 'New Delhi', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [28]:
stop_points = ML['stop_points'].value_counts().index
stop_points

Index(['[' BOM ']', '[]', '[' DEL ']', '[' HYD ']', '[' MAA ']', '[' BLR ']',
       '[' JAI ', ' BOM ']', '[' AMD ', ' BOM ']', '[' IDR ', ' BOM ']',
       '[' HYD ', ' MAA ']', '[' PNQ ']', '[' BBI ']', '[' AMD ']',
       '[' BHO ', ' BOM ']', '[' IXR ', ' DEL ']', '[' HYD ', ' BOM ']',
       '[' NAG ', ' BOM ']', '[' CCU ', ' BOM ']', '[' GOI ', ' BOM ']',
       '[' GAU ']', '[' BBI ', ' BOM ']', '[' LKO ', ' BOM ']',
       '[' ATQ ', ' BOM ']', '[' GAU ', ' DEL ']', '[' JDH ', ' BOM ']',
       '[' BOM ', ' COK ']', '[' DEL ', ' COK ']', '[' BDQ ', ' BOM ']',
       '[' DEL ', ' AMD ']', '[' IXB ']', '[' LKO ']', '[' UDR ', ' BOM ']',
       '[' TRV ']', '[' RPR ', ' NAG ', ' BOM ']', '[' BOM ', ' GOI ']',
       '[' NAG ']', '[' BOM ', ' AMD ']', '[' IXU ', ' BOM ']',
       '[' IXC ', ' BOM ']', '[' CCU ']', '[' VNS ', ' DEL ']',
       '[' MAA ', ' BOM ']', '[' KNU ']', '[' PAT ']', '[' BOM ', ' BHO ']',
       '[' BBI ', ' HYD ']', '[' COK ']', '[' GWL ', ' IDR ', ' BOM ']

# Modeling

## Detecting which best model

In [29]:
scalers = [
    ('std scaler:', StandardScaler()), 
    ('MinMax scaler :', MinMaxScaler()), 
    ('Robust scaler: ', RobustScaler())
]

In [30]:
#models
models = [
    ('LinearR',LinearRegression()),      
    ('KNN', KNeighborsRegressor()),
    ('SVR', SVR()),

    ('DTR', DecisionTreeRegressor()),
    ('RFR', RandomForestRegressor()),
    ('ADA' ,AdaBoostRegressor()),
    ('GB',GradientBoostingRegressor()),
    
    ('XGB', XGBRegressor())]
    

In [31]:
# metrics and scores names
(sklearn.metrics.get_scorer_names())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [32]:
result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() ,binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)
            ],
        remainder='passthrough'
         )
    
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append((scaler_name, scaler))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'R2':'r2'}
#                                                        'neg_mean_squared_error': 'neg_mean_squared_error',
#                                                        'neg_mean_absolute_error':'neg_mean_absolute_error',
#                                                        'neg_root_mean_squared_error':'neg_root_mean_squared_error',
#                                                                       }# interested scors and metrics
                                                             
                             , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets
        
        
        # Access the results
        test_r2 = round(cv_results['test_R2'].mean() , 5)
        train_r2 = round(cv_results['train_R2'].mean() , 5)
        print("test_R^2 :", test_r2)
        print("train_R^2:", train_r2)
        print('-'*50)

        result_models.append([test_r2,train_r2])
       # print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For LinearR:
test_R^2 : 0.62754
train_R^2: 0.63558
--------------------------------------------------
For KNN:
test_R^2 : 0.79184
train_R^2: 0.86373
--------------------------------------------------
For SVR:
test_R^2 : 0.03115
train_R^2: 0.0319
--------------------------------------------------
For DTR:
test_R^2 : 0.84257
train_R^2: 0.99497
--------------------------------------------------
For RFR:
test_R^2 : 0.89914
train_R^2: 0.9814
--------------------------------------------------
For ADA:
test_R^2 : 0.31216
train_R^2: 0.33816
--------------------------------------------------
For GB:
test_R^2 : 0.82141
train_R^2: 0.83589
--------------------------------------------------
For XGB:
test_R^2 : 0.90362
train_R^2: 0.96736
--------------------------------------------------
****************************************************************************************************
________________________________________________________________________________________________

In [33]:
result = pd.DataFrame(data=result_models,index=[
    'LinearR_std','KNN_std','SVR_std','DTR_std','RFR_std','ADA_std' ,'GB_std','XGB_std',
    'LinearR_minmax','KNN_minmax','SVR_minmax','DTR_minmax','RFR_minmax','ADA_minmax' ,'GB_minmax','XGB_minmax',
    'LinearR_robust','KNN_robust','SVR_robust','DTR_robust','RFR_robust','ADA_robust' ,'GB_robust','XGB_robust'],
                      columns=['test_R2' , 'train_R2'] )

In [34]:
result

,test_R2,train_R2
LinearR_std,0.62754,0.63558
KNN_std,0.79184,0.86373
SVR_std,0.03115,0.03190
DTR_std,0.84257,0.99497
RFR_std,0.89914,0.98140
ADA_std,0.31216,0.33816
GB_std,0.82141,0.83589
XGB_std,0.90362,0.96736
LinearR_minmax,0.62693,0.63467
KNN_minmax,0.78916,0.86813


In [35]:
result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() ,binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)
            ],
        remainder='passthrough'
         )
    
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append((scaler_name, scaler))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'R2':'r2',
                                                               'neg_mean_squared_error': 'neg_mean_squared_error',
                                                        'neg_mean_absolute_error':'neg_mean_absolute_error',
                                                        'neg_root_mean_squared_error':'neg_root_mean_squared_error'
                                                                       }# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets
        
        
        # Access the results
        test_r2 = cv_results['test_R2'].mean().round(5)
        train_r2 = cv_results['train_R2'].mean().round(5)
        print("test_R^2 :", test_r2)
        print("train_R^2:", train_r2)
        print('-'*50)
        test_neg_mean_squared_error = cv_results['test_neg_mean_squared_error'].mean().round(5)
        train_neg_mean_squared_error = cv_results['train_neg_mean_squared_error'].mean().round(5)
        print("test_neg_mean_squared_error :", test_neg_mean_squared_error)
        print("train_neg_mean_squared_error:", train_neg_mean_squared_error)
        print('-'*50)
        test_neg_mean_absolute_error = cv_results['test_neg_mean_absolute_error'].mean().round(5)
        train_neg_mean_absolute_error = cv_results['train_neg_mean_absolute_error'].mean().round(5)
        print("test_neg_mean_absolute_error :", test_neg_mean_absolute_error)
        print("train_neg_mean_absolute_error:", train_neg_mean_absolute_error)
        print('-'*50)
        test_neg_root_mean_squared_error = cv_results['test_neg_root_mean_squared_error'].mean().round(5)
        train_neg_root_mean_squared_error = cv_results['train_neg_root_mean_squared_error'].mean().round(5)
        print("test_neg_root_mean_squared_error :", test_neg_root_mean_squared_error)
        print("train_neg_root_mean_squared_error:", train_neg_root_mean_squared_error)
        print('-'*50)

        result_models.append([test_r2 , train_r2 , test_neg_mean_squared_error , train_neg_mean_squared_error ,
                              test_neg_mean_absolute_error , train_neg_mean_absolute_error ,
                              test_neg_root_mean_squared_error , train_neg_root_mean_squared_error])
        print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For LinearR:
test_R^2 : 0.62754
train_R^2: 0.63558
--------------------------------------------------
test_neg_mean_squared_error : -7827909.92493
train_neg_mean_squared_error: -7676263.92616
--------------------------------------------------
test_neg_mean_absolute_error : -1943.0399
train_neg_mean_absolute_error: -1929.07117
--------------------------------------------------
test_neg_root_mean_squared_error : -2796.53674
train_neg_root_mean_squared_error: -2770.16441
--------------------------------------------------
**************************************************
For KNN:
test_R^2 : 0.79184
train_R^2: 0.86373
--------------------------------------------------
test_neg_mean_squared_error : -4385062.95831
train_neg_mean_squared_error: -2870819.76212
--------------------------------------------------
test_neg_mean_absolute_error : -1072.80305
train_neg_mean_absolute_error: -846.01194
--------------------------------------------------
test_neg_root_mean_squared_error

test_R^2 : 0.82034
train_R^2: 0.83589
--------------------------------------------------
test_neg_mean_squared_error : -3778221.53104
train_neg_mean_squared_error: -3456963.80423
--------------------------------------------------
test_neg_mean_absolute_error : -1297.98558
train_neg_mean_absolute_error: -1266.10973
--------------------------------------------------
test_neg_root_mean_squared_error : -1943.28899
train_neg_root_mean_squared_error: -1859.19395
--------------------------------------------------
**************************************************
For XGB:
test_R^2 : 0.90362
train_R^2: 0.96736
--------------------------------------------------
test_neg_mean_squared_error : -2033750.45674
train_neg_mean_squared_error: -687540.51117
--------------------------------------------------
test_neg_mean_absolute_error : -751.49673
train_neg_mean_absolute_error: -515.88694
--------------------------------------------------
test_neg_root_mean_squared_error : -1418.09156
train_neg_root_me

In [36]:
scores = pd.DataFrame(data=result_models,index=[
    'LinearR_std','KNN_std','SVR_std','DTR_std','RFR_std','ADA_std' ,'GB_std','XGB_std',
    'LinearR_minmax','KNN_minmax','SVR_minmax','DTR_minmax','RFR_minmax','ADA_minmax' ,'GB_minmax','XGB_minmax',
    'LinearR_robust','KNN_robust','SVR_robust','DTR_robust','RFR_robust','ADA_robust' ,'GB_robust','XGB_robust'],
                      columns=['test_R2' , 'train_R2','test_neg_mean_squared_error' , 'train_neg_mean_squared_error' ,
                              'test_neg_mean_absolute_error' , 'train_neg_mean_absolute_error' ,
                              'test_neg_root_mean_squared_error' , 'train_neg_root_mean_squared_error'] )

In [37]:
scores

,test_R2,train_R2,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error
LinearR_std,0.62754,0.63558,-7.827910e+06,-7.676264e+06,-1943.03990,-1929.07117,-2796.53674,-2770.16441
KNN_std,0.79184,0.86373,-4.385063e+06,-2.870820e+06,-1072.80305,-846.01194,-2091.66810,-1694.20914
SVR_std,0.03115,0.03190,-2.040873e+07,-2.039511e+07,-3451.70509,-3449.42704,-4516.16301,-4516.02898
DTR_std,0.82544,0.99497,-3.679144e+06,-1.058085e+05,-760.35220,-52.16883,-1908.74560,-325.02814
RFR_std,0.89801,0.98180,-2.147338e+06,-3.835324e+05,-665.01985,-268.08898,-1458.76640,-619.05303
ADA_std,0.28618,0.31968,-1.502039e+07,-1.432628e+07,-3143.33228,-3103.40525,-3865.79197,-3773.80788
GB_std,0.82199,0.83589,-3.742406e+06,-3.456964e+06,-1295.97564,-1266.10973,-1933.98630,-1859.19395
XGB_std,0.90362,0.96736,-2.033750e+06,-6.875405e+05,-751.49673,-515.88694,-1418.09156,-828.63326
LinearR_minmax,0.62693,0.63467,-7.841191e+06,-7.695306e+06,-1951.38379,-1935.87828,-2798.96553,-2773.50494
KNN_minmax,0.78916,0.86813,-4.453885e+06,-2.778533e+06,-1071.20605,-833.85497,-2104.66977,-1666.41871


## HyperParameter Tuning 

In [38]:
# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [39]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('XGB', XGBRegressor()))
pipee=Pipeline(steps=steps)

In [40]:
param_grid = {
 'XGB__learning_rate':[0.1],
 'XGB__n_estimators':[800,700,1000],
 'XGB__max_depth':[6],
 'XGB__min_child_weight':[3],
 'XGB__gamma':[4,3,2]}
#  'XGB__subsample':[0.8],
#  'XGB__colsample_bytree':[0.8],
#  'XGB__objective' :['binary:logistic'],
#  'XGB__nthread':[4],
#  'XGB__scale_pos_weight':[1],
#  'XGB__seed':[27]}

In [41]:
param_grid = {
 'XGB__learning_rate':[0.1],
 'XGB__n_estimators':[800,700,1000,1200,1300],
 'XGB__max_depth':[6],
 'XGB__min_child_weight':[3],
 'XGB__gamma':[2,3,4],#}#,4]#,5,6]}
  'XGB__subsample':[2,1,.8],
  'XGB__colsample_bytree':[0.8]}#,
#  'XGB__objective' :['binary:logistic'],
#  'XGB__nthread':[4],
#  'XGB__scale_pos_weight':[1],
#  'XGB__seed':[27]}'''

In [42]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipee, param_grid=param_grid, cv=5, scoring='r2'
                                                             , n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'XGB__colsample_bytree': 0.8, 'XGB__gamma': 2, 'XGB__learning_rate': 0.1, 'XGB__max_depth': 6, 'XGB__min_child_weight': 3, 'XGB__n_estimators': 700, 'XGB__subsample': 1}
Best Score: 0.9150320689093956


In [43]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.9150320689093956
Mean Train Score: 0.9768828977590027


# Saving model with its best parameters

In [44]:
Model = grid_search.best_estimator_

In [45]:
Model.fit(X,y)

Pipeline(steps=[('encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('binary_encoder',
                                                  BinaryEncoder(),
                                                  ['Airline', 'stop_points']),
                                                 ('onh_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Source', 'Destination',
                                                   'dep_day_part',
                                                   'arrival_day_part'])])),
                ('scaler', RobustScaler()),
                ('XGB',
                 XGBRegressor(base_score=None, booster=None, call...
                              feature_types=None, gamma=2, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None, min_child_weight=3,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=700,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

# scoring best params

In [46]:
# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder().set_output(transform="pandas") , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [47]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('XGB', XGBRegressor(max_depth=6 , min_child_weight=3 , gamma=2 , n_estimators=700 ,
                                 subsample= 1, colsample_bytree= 0.8, learning_rate=.1)))
pipe=Pipeline(steps=steps)

In [48]:
cv_results = cross_validate(pipe, X, y, scoring={'R2':'r2',
                                                'neg_mean_squared_error': 'neg_mean_squared_error',
                                                'neg_mean_absolute_error':'neg_mean_absolute_error',
                                                'neg_root_mean_squared_error':'neg_root_mean_squared_error'}# interested scors and metrics

                                , cv=5, return_train_score=True,n_jobs=-1)

In [49]:
test_r2 = cv_results['test_R2'].mean().round(5)
train_r2 = cv_results['train_R2'].mean().round(5)
print("test_R^2 :", test_r2)
print("train_R^2:", train_r2)
print('-'*50)
test_neg_mean_squared_error = cv_results['test_neg_mean_squared_error'].mean().round(5)
train_neg_mean_squared_error = cv_results['train_neg_mean_squared_error'].mean().round(5)
print("test_neg_mean_squared_error :", test_neg_mean_squared_error)
print("train_neg_mean_squared_error:", train_neg_mean_squared_error)
print('-'*50)
test_neg_mean_absolute_error = cv_results['test_neg_mean_absolute_error'].mean().round(5)
train_neg_mean_absolute_error = cv_results['train_neg_mean_absolute_error'].mean().round(5)
print("test_neg_mean_absolute_error :", test_neg_mean_absolute_error)
print("train_neg_mean_absolute_error:", train_neg_mean_absolute_error)
print('-'*50)
test_neg_root_mean_squared_error = cv_results['test_neg_root_mean_squared_error'].mean().round(5)
train_neg_root_mean_squared_error = cv_results['train_neg_root_mean_squared_error'].mean().round(5)
print("test_neg_root_mean_squared_error :", test_neg_root_mean_squared_error)
print("train_neg_root_mean_squared_error:", train_neg_root_mean_squared_error)
print('-'*50)


test_R^2 : 0.91503
train_R^2: 0.97688
--------------------------------------------------
test_neg_mean_squared_error : -1790605.70471
train_neg_mean_squared_error: -486893.71696
--------------------------------------------------
test_neg_mean_absolute_error : -696.36589
train_neg_mean_absolute_error: -405.91562
--------------------------------------------------
test_neg_root_mean_squared_error : -1335.88854
train_neg_root_mean_squared_error: -697.16157
--------------------------------------------------


* **These are best scores for the Model**

## Saving important objects For Deploying

In [50]:
jb.dump(Model,"Model_.pkl")

['Model_.pkl']

In [51]:
jb.dump(X.columns,"Columns.pkl")

['Columns.pkl']

In [52]:
jb.dump(Airlines ,"Airlines.pkl")
jb.dump(Sources ,"Sources.pkl")
#jb.dump(Destinations ,"Destinations.pkl")
jb.dump(Source_dest ,"Source_dest.pkl")
jb.dump(stop_points ,"stop_points.pkl")


['stop_points.pkl']

<a href="https://airlineflightpriceprediction.streamlit.app/">Flight_Price_prediction_app </a>